# **Estruturas criptograficas: TP2 problema 3**

In [26]:
import hashlib, os
from sage.all import *

In [27]:

# # Geração dos primos  q, p
# bq     = 192                # tamanho em bits do primo "q"
# bp     = 768                # tamanho minimo em bits do primo "p"

# q = random_prime(2^bq-1,lbound=2^(bq-1))

# t = q*3*2^(bp - bq)
# while not is_prime(t-1):
#     t = t << 1

# p = t - 1

# # Aneis e Corpos
# Fp     = GF(p)                  # corpo primo com "p" elementos
# R.<z>  = Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
# f     = R(z^2 + z + 1)
# Fp2.<z> = GF(p^2, modulus=f)   
# # extensão de Fp de dimensão 2 cujo módulo é o polinómio "f"
# # o polinómio "f"  é irredutivel, tem grau 2 e verifica  z^3 = 1 mod f
# # se o ponto (x,y) verificar a equação y^2 = x^3 + 1, 
# #      então o ponto (z*x,y) verifica a mesma equação

# def trace(x):       # função linear que mapeia Fp2  em  Fp
#     return x + x^p

# # Curvas Elipticas supersingulares em Sagemath

# # a curva supersingular sobre Fp2  definida pela equação  y^2 = x^3 + 1
# E2 = EllipticCurve(Fp2, [0,1])

# # ponto arbitrário  de ordem "q" em E2        
# cofac = (p + 1)//q
# G = cofac * E2.random_point()

# # emparelhamento e oraculo DDHP

# def phi(P):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
#     (x,y) = P.xy()
#     return E2(z*x,y)

# def TateX(P,Q,l=1):      # o emparelhamento de Tate generalizado
#     return P.tate_pairing(phi(Q), q, 2)^l

# def ddhp(P,Q,R):        # o oraculo DDHP  que decide se (P,Q,R) é um triplo de DH
#     return tateX(P,Q) == tateX(R,G)

In [28]:
class BF:
    
    def __init__(self):
        self.version = 2
        self.n = 1024
        self.n_p = 512
        self.n_q = 160
        self.hashfnc = hashlib.sha1
        
        self.q = random_prime(2^self.n_q-1,lbound=2^(self.n_q-1))
                
        # print("q:" + str(q))
        
        #NOTE: verificar se está certo
        t = self.q*3*2^(self.n_p - self.n_q)
        while not is_prime(t-1):
            t = t << 1

        self.p = t - 1
        
        # print("p:" + str(p))
        
        self.Fp     = GF(self.p)                  # corpo primo com "p" elementos
        R.<z>  = self.Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
        f     = R(z^2 + z + 1)
        Fp2.<z> = GF(self.p^2, modulus=f)   
        self.E2 = EllipticCurve(Fp2, [0,1])

        # ponto arbitrário  de ordem "q" em E2        
        cofac = (self.p + 1)// self.q
        self.G = cofac * self.E2.random_point()
        # print("G:" + str(G.xy()))

        
    def ID(self, id):
        id_int = self.h(id)
        return self.g(id_int)
        
    def g(self, n):
        return n * self.G
    
    def h(self, a):
        return int.from_bytes(a.encode(), byteorder='big')
    
    def H(self, a):
        return mod(a, self.q)
    
    def KeyGen(self):
        s = 0
        while s <= 2:
            nounce = os.urandom(16)
            s = self.hashfnc(nounce).digest()
            s = int.from_bytes(s, byteorder='big')
            s = s % self.q
        
        beta = self.g(s)
                
        return s, beta
    
    def KeyExtract(self, id, s):
        d = self.ID(id)
        
        key = s * d

        return key    
    
    def phi(self, P):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
        R.<z>  = self.Fp[]                   
        f     = R(z^2 + z + 1)
        Fp2.<z> = GF(self.p^2, modulus=f) 
        
        (x,y) = P.xy()
        return self.E2(z*x,y)
    
    def TateX(self, P, Q, l=1):      # o emparelhamento de Tate generalizado
        return P.tate_pairing(self.phi(Q), self.q, 2)^l 

    def trace(self, x):       # função linear que mapeia Fp2  em  Fp
        return x + x^self.p    
        
    def Encrypt(self, beta, id, x):
        x = self.h(x)
        
        #IN
        d = self.ID(id)
        
        nounce = os.urandom(16)
        v = self.hashfnc(nounce).digest()
        v = int.from_bytes(v, byteorder='big')
        v = v % self.q
        
        a = self.H(v ^^ x)
        
        mu = self.TateX(beta, d, a)

        
        #OUT
        alfa = self.g(a)
        
        v_ = self.trace(mu)
        v_ = int(v) ^^ int(v_)

        x_ = int(x) ^^ int(self.H(v))
        
        print("Encryption finished!")
        
        return alfa, v_, x_
        
    
    def Decrypt(self, alfa, v_, x_, key):
        
        #IN
        mu = self.TateX(alfa, key)
        
        v = self.trace(mu)
        v = int(v_) ^^ int(v)
        
        x = int(x_) ^^ int(self.H(v))
        
        #OUT
        a = self.H(v ^^ x)
        
        if alfa == self.g(a):
            m = (int(x).to_bytes((int(x.bit_length()) + 7) // 8, byteorder='big')).decode()
            print("Decryption successful!\nMessage: " + m)
            return m
        else:
            print("Decryption failed!")
            return None
    
       

In [29]:
bf = BF()

s, beta = bf.KeyGen()

alfa, v_, x_ = bf.Encrypt(beta, "Universidade do Minho", "SageMath ser merda")


Encryption finished!


In [33]:
key = bf.KeyExtract("Universidade do Minho", s)

m = bf.Decrypt(alfa, v_, x_, key)

Decryption successful!
Message: SageMath ser merda


In [34]:
key = bf.KeyExtract("Universidade de Coimbra", s)

m = bf.Decrypt(alfa, v_, x_, key)

Decryption failed!


In [32]:
# def Encrypt(self, E, p, q, G, G_pub, id, m):
#         hashlen = self.hashfnc().digest_size
#         print("hashlen:" + str(hashlen))
        
#         Q_id = self.HashToPoint(E, p, q, id)
        
#         rho = os.urandom(16)
#         rho = self.hashfnc(rho).digest()
        
#         print("rho:" + str(len(rho)))
        
#         t = self.hashfnc(m.encode()).digest()
        
#         print("t:" + str(len(t)))
        
#         l = self.HashToRange(str(rho + t), q)  
        
#         print(l)
        
#         U = l * G
        
#         print("U:" + str(U.xy()))
        
#         theta = self.TateX(p, q, G_pub, Q_id, E)

#         print("theta:" + str(theta))
        
#         theta_ = pow(theta, l)
        
#         print("theta_:" + str(theta_))

# def HashToPoint(self, id, y = 0):
#     if y == 0:
#         y = self.HashToRange(id, self.p)
#     else:
#         y = mod(id, self.p)
#     x = pow((y^2 - 1),((2 * self.p - 1) // 3), self.p)
    
#     Q_ = self.E2.point((x, y))
    
#     Q = ((self.p + 1) // self.q) * Q_
    
#     return Q
    
# def HashToRange(self, s, n):
#     hashlen = self.hashfnc().digest_size
#     v = [0, 0, 0]
#     h = ['', '', '']
    
#     v[0] = 0
#     h[0] = b'\x00' * hashlen
#     for i in range(1, 3):
#         t = h[i-1] + s.encode()
#         h[i] = self.hashfnc(t).digest()
#         a = int.from_bytes(h[i], byteorder='big')
#         v[i] = (256**hashlen) * v[i-1] + a

#     v = v[i] % n
#     return v
        